In [37]:
import pandas as pd
import numpy as np 

# reading train data
train_data = pd.read_csv('pp5i_train.gr.csv')

In [38]:
# droping the index column
train_data.drop(['SNO'], axis=1, inplace=True)

#  Threshold data
train_data[train_data>16000] = 16000
train_data[train_data<20] = 20

In [39]:
# calculating and removing fold differences
train_data['fold_differences'] = train_data.max(axis=1) / train_data.min(axis=1)
train_data.drop(train_data[ train_data['fold_differences']<2 ].index, axis=0, inplace=True)
train_data.drop(['fold_differences'], axis=1, inplace=True)


In [40]:
# reading class labels and adding them to the train data
labels = pd.read_table('pp5i_train_class.txt')
train_data.iloc[-1, :] = labels.T.values
for idx, item in enumerate(train_data.iloc[-1, :]):
    train_data.iloc[-1, idx] = item[0]

train_data = train_data.T



train_data

,0,1,2,3,4,5,6,7,8,9,...,7056,7057,7058,7059,7060,7061,7063,7065,7066,7068
1,30,22,29,76,167,32,20,165,47,101,...,27,20,155,240,27,161,34,116,44,MED
2,46,31,70,208,211,20,20,184,36,130,...,129,20,143,213,27,265,98,123,43,MED
3,31,20,20,244,179,229,20,219,41,131,...,29,20,120,367,24,126,21,142,44,MED
4,31,20,20,39,119,20,20,156,26,86,...,59,20,161,358,27,163,20,134,38,MED
5,26,26,20,85,161,34,20,206,34,140,...,97,20,119,373,41,138,29,153,42,MED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,68,21,35,30,131,20,22,140,37,81,...,51,21,106,181,20,364,34,297,39,JPA
66,77,22,30,31,132,20,20,150,43,128,...,31,26,127,177,26,251,45,267,42,JPA
67,56,25,65,31,158,20,27,173,54,108,...,39,20,73,159,20,315,27,290,36,JPA
68,41,21,32,27,164,20,20,165,74,121,...,37,20,83,137,21,248,46,255,44,JPA


## computing T-Test for MED class

In [41]:
med = train_data[ train_data.iloc[:, -1] == "MED" ]
med = med.T
med.drop(med.index[-1], inplace=True)
# med = med.T

med_inverse = train_data[ train_data.iloc[:, -1] != "MED"]
med_inverse = med_inverse.T
med_inverse.drop(med_inverse.index[-1], inplace=True)
# med_inverse = med_inverse.T


# print(med.shape, med_inverse.shape)

med["mean"] = med.mean(axis=1, skipna=True)
med["std"] = med.std(axis=1, skipna=True)
# print(med.head(3))

med_inverse["mean"] = med_inverse.mean(axis=1, skipna=True)
med_inverse["std"] = med_inverse.std(axis=1, skipna=True)
# print(med_inverse.head(3))

nume = med.loc[:,'mean'] - med_inverse.loc[:,'mean']
deno = np.sqrt( med.loc[:,'std'].pow(2)/39  + med_inverse.loc[:,'std'].pow(2)/30)

med.loc[:, 'T'] = nume / deno

med.drop(labels=['mean', 'std'], axis=1, inplace=True)


med.sort_values(['T'], ascending=False, na_position='last', inplace=True)
med['MED'] = "MED"
med.drop(labels='T', axis=1, inplace=True)
med.iloc[:30, :]

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,MED
2773,931,845,1107,1187,969,1172,1010,588,928,469,...,682,747,407,699,874,872,848,1069,786,MED
1868,724,579,608,619,676,651,751,243,927,927,...,1106,842,573,576,696,791,631,819,751,MED
3704,179,192,215,187,263,233,252,173,200,154,...,147,137,189,148,204,208,185,171,200,MED
528,592,317,387,312,370,278,281,197,199,291,...,100,198,300,418,320,385,353,159,258,MED
404,989,1120,1232,1132,1030,814,703,1034,1321,716,...,982,799,1669,883,1243,1463,848,1087,1150,MED
5132,160,172,184,164,157,158,186,184,124,159,...,162,114,216,149,191,122,210,222,250,MED
2305,628,760,596,825,536,471,165,396,192,213,...,133,116,124,554,1060,1142,577,773,332,MED
6411,75,59,62,88,65,84,67,105,66,64,...,74,42,47,39,39,72,138,38,63,MED
6402,479,433,799,588,939,790,658,458,1021,496,...,407,541,373,620,465,537,648,662,1030,MED
2954,1274,1308,1393,1075,1284,1341,1222,849,1414,348,...,1352,1281,687,626,810,1330,1082,1614,1094,MED


## computing T-Test for MGL class

In [47]:
mgl = train_data[ train_data.iloc[:, -1] == "MGL" ]
mgl = mgl.T
mgl.drop(mgl.index[-1], inplace=True)
# mgl = mgl.T

mgl_inverse = train_data[ train_data.iloc[:, -1] != "MGL"]
mgl_inverse = mgl_inverse.T
mgl_inverse.drop(mgl_inverse.index[-1], inplace=True)
# mgl_inverse = mgl_inverse.T

# print(mgl.shape, mgl_inverse.shape)

mgl["mean"] = mgl.mean(axis=1, skipna=True)
mgl["std"] = mgl.std(axis=1, skipna=True)
# print(mgl.head(3))

mgl_inverse["mean"] = mgl_inverse.mean(axis=1, skipna=True)
mgl_inverse["std"] = mgl_inverse.std(axis=1, skipna=True)
# print(mgl_inverse.head(3))


nume = mgl.loc[:,'mean'] - mgl_inverse.loc[:,'mean']
deno = np.sqrt( mgl.loc[:,'std'].pow(2)/39  + mgl_inverse.loc[:,'std'].pow(2)/30)

mgl.loc[:, 'T'] = nume / deno

mgl.drop(labels=['mean', 'std'], axis=1, inplace=True)
mgl.sort_values(['T'], ascending=False, na_position='last', inplace=True)
mgl['MGL'] = "MGL"
mgl.drop(labels='T', axis=1, inplace=True)
mgl.iloc[:30, :]
mgl.reset_index(inplace=True )
mgl


,index,40,41,42,43,44,45,46,MGL
0,3309,110,98,118,56,118,98,123,MGL
1,4168,114,94,98,77,103,95,92,MGL
2,3959,69,61,74,60,70,66,69,MGL
3,1728,209,189,198,188,172,176,172,MGL
4,2175,331,272,348,208,286,317,278,MGL
...,...,...,...,...,...,...,...,...,...
6407,2756,39,74,39,78,65,34,46,MGL
6408,4377,38,29,34,64,51,44,35,MGL
6409,3760,60,52,23,72,57,20,41,MGL
6410,2264,88,98,99,163,92,63,106,MGL


## computing RHB class T-Test

In [19]:
import numpy as np 


rhb = train_data[ train_data.iloc[:, -1] == "RHB" ]
rhb = rhb.T
rhb.drop(rhb.index[-1], inplace=True)
# rhb = rhb.T

rhb_inverse = train_data[ train_data.iloc[:, -1] != "RHB"]
rhb_inverse = rhb_inverse.T
rhb_inverse.drop(rhb_inverse.index[-1], inplace=True)
# rhb_inverse = rhb_inverse.T

print(rhb.shape, rhb_inverse.shape)

rhb["mean"] = rhb.mean(axis=1, skipna=True)
rhb["std"] = rhb.std(axis=1, skipna=True)
# print(rhb.head(3))

rhb_inverse["mean"] = rhb_inverse.mean(axis=1, skipna=True)
rhb_inverse["std"] = rhb_inverse.std(axis=1, skipna=True)
# print(rhb_inverse.head(3))

nume = rhb.loc[:,'mean'] - rhb_inverse.loc[:,'mean']
deno = np.sqrt( rhb.loc[:,'std'].pow(2)/39  + rhb_inverse.loc[:,'std'].pow(2)/30)

rhb.loc[:, 'T'] = nume / deno
rhb.drop(labels=['mean', 'std'], axis=1, inplace=True)
rhb

(6412, 7) (6412, 62)


,47,48,49,50,51,52,53,mean,std,T
0,81,64,71,35,41,54,55,57.285714,14.992515,3.695682
1,33,45,39,27,25,26,27,31.714286,7.105617,1.242845
2,87,126,28,20,39,20,31,50.142857,37.627606,1.940360
3,23,20,20,33,219,41,344,100.000000,119.759282,0.652660
4,190,250,237,179,174,131,237,199.714286,40.027542,0.297334
...,...,...,...,...,...,...,...,...,...,...
7060,20,22,25,22,21,20,20,21.428571,1.678191,-1.776405
7061,163,190,114,112,337,744,234,270.571429,206.192599,2.544185
7063,30,20,20,20,22,28,30,24.285714,4.463000,-2.141304
7065,134,128,127,90,99,96,74,106.857143,21.155402,-4.742747


## computing EPD class T-Test

In [20]:


epd = train_data[ train_data.iloc[:, -1] == "EPD" ]
epd = epd.T
epd.drop(epd.index[-1], inplace=True)
# epd = epd.T

epd_inverse = train_data[ train_data.iloc[:, -1] != "EPD"]
epd_inverse = epd_inverse.T
epd_inverse.drop(epd_inverse.index[-1], inplace=True)
# epd_inverse = epd_inverse.T

print(epd.shape, epd_inverse.shape)

epd["mean"] = epd.mean(axis=1, skipna=True)
epd["std"] = epd.std(axis=1, skipna=True)
# print(epd.head(3))

epd_inverse["mean"] = epd_inverse.mean(axis=1, skipna=True)
epd_inverse["std"] = epd_inverse.std(axis=1, skipna=True)
# print(epd_inverse.head(3))

nume = epd.loc[:,'mean'] - epd_inverse.loc[:,'mean']
deno = np.sqrt( epd.loc[:,'std'].pow(2)/39  + epd_inverse.loc[:,'std'].pow(2)/30)

epd.loc[:, 'T'] = nume / deno
epd.drop(labels=['mean', 'std'], axis=1, inplace=True)
epd

(6412, 10) (6412, 59)


,54,55,56,57,58,59,60,61,62,63,mean,std,T
0,51,34,40,44,27,42,35,49,31,51,40.4,8.077128,-0.765436
1,25,20,24,30,25,21,38,30,22,24,25.9,5.127377,-2.128768
2,23,219,24,32,25,45,20,205,20,61,67.4,73.388282,3.106462
3,251,90,62,164,100,188,55,203,152,53,131.8,66.045136,2.584905
4,173,164,125,174,142,207,137,180,107,147,155.6,28.022134,-4.698209
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7060,20,20,22,24,24,20,31,20,25,20,22.6,3.382307,-0.851323
7061,170,141,167,170,308,279,410,238,718,374,297.5,165.334963,4.112878
7063,25,23,26,25,25,51,101,25,39,56,39.6,23.414525,2.152798
7065,221,148,222,113,96,170,117,203,136,118,154.4,44.540319,-2.730792


## computing JPA class T-Test

In [21]:

jpa = train_data[ train_data.iloc[:, -1] == "JPA" ]
jpa = jpa.T
jpa.drop(jpa.index[-1], inplace=True)
# jpa = jpa.T

jpa_inverse = train_data[ train_data.iloc[:, -1] != "JPA"]
jpa_inverse = jpa_inverse.T
jpa_inverse.drop(jpa_inverse.index[-1], inplace=True)
# jpa_inverse = jpa_inverse.T

print(jpa.shape, jpa_inverse.shape)

jpa["mean"] = jpa.mean(axis=1, skipna=True)
jpa["std"] = jpa.std(axis=1, skipna=True)
# print(jpa.head(3))

jpa_inverse["mean"] = jpa_inverse.mean(axis=1, skipna=True)
jpa_inverse["std"] = jpa_inverse.std(axis=1, skipna=True)
# print(jpa_inverse.head(3))

nume = jpa.loc[:,'mean'] - jpa_inverse.loc[:,'mean']
deno = np.sqrt( jpa.loc[:,'std'].pow(2)/39  + jpa_inverse.loc[:,'std'].pow(2)/30)

jpa.loc[:, 'T'] = nume / deno
jpa.drop(labels=['mean', 'std'], axis=1, inplace=True)
jpa

(6412, 6) (6412, 63)


,64,65,66,67,68,69,mean,std,T
0,71,68,77,56,41,38,58.500000,14.840822,3.966327
1,20,21,22,25,21,20,21.500000,1.707825,-5.063769
2,62,35,30,65,32,25,41.500000,15.861379,1.021336
3,28,30,31,31,27,20,27.833333,3.804237,-3.356653
4,170,131,132,158,164,172,154.500000,16.869598,-4.768212
...,...,...,...,...,...,...,...,...,...
7060,20,20,26,20,21,20,21.166667,2.192158,-1.942311
7061,198,364,251,315,248,285,276.833333,52.969855,4.212755
7063,26,34,45,27,46,61,39.833333,12.266712,2.589455
7065,234,297,267,290,255,397,290.000000,52.262160,2.693078


In [6]:
# reading test data and Threshold it 
test_data = pd.read_csv('pp5i_test.gr.csv')
test_data.drop(['SNO'], axis=1, inplace=True)
test_data[test_data>16000] = 16000
test_data[test_data<20] = 20